In [599]:
import numpy as np
import scipy.optimize as opt
import pandas as pd
from sklearn import datasets

### Задание

1. Загрузите данные. Используйте датасет с ирисами. Его можно загрузить непосредственно из библиотеки Sklearn. В данных оставьте только 2 класса: Iris Versicolor, Iris Virginica.


2. Самостоятельно реализуйте логистическую регрессию, без использования метода LogisticRegression из библиотеки. Можете использовать библиотеки pandas, numpy, math для реализации. Оформите в виде функции. *Оформите в виде класса с методами.


3. Реализуйте метод градиентного спуска. Обучите логистическую регрессию этим методом. Выберете и посчитайте метрику качества. Метрика должна быть одинакова для всех пунктов домашнего задания. Для упрощения сравнения выберете только одну метрику.


4. Повторите п. 3 для метода скользящего среднего (Root Mean Square Propagation, RMSProp).


5. Повторите п. 3 для ускоренного по Нестерову метода адаптивной оценки моментов (Nesterov–accelerated Adaptive Moment Estimation, Nadam).


6. Сравните значение метрик для реализованных методов оптимизации. Можно оформить в виде таблицы вида |метод|метрика|время работы| (время работы опционально). Напишите вывод.

In [600]:
# фиксируем значение случайности порядка строк
np.random.seed(1)

#Заводим датасет в датафрейм
iris_df = pd.DataFrame(iris.data)
iris_df.columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
iris_df['target'] = pd.DataFrame(iris.target)
two_class = iris_df[(iris_df['target'] == 1) | (iris_df['target'] == 2)] 



Опытным путем было установлено, что если оставить target  в виде 1 и 2, то качество обучения резко снижается.А вот если вместо 1 и 2 будет 0 и 1, то все меняется.

In [601]:
mapping = {2:1,1:0}
two_class['target'] = two_class['target'].map(mapping)

<ipython-input-601-07cba915276b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  two_class['target'] = two_class['target'].map(mapping)


In [603]:
#Проверяем, какие у нас теперь есть классы:
two_class['target'].unique()

array([0, 1], dtype=int64)

In [604]:
# вот тут загадка лично для меня. если не сделать перемешивание порядка строк, то обучение не работает.
data  = two_class.sample(frac=1)
X = data.iloc[:,:-1].values
y = data.iloc[:,-1].values
y = y[:,np.newaxis]

In [605]:
# Разделяем наши данные на тренировочную и тестовую выборки (80% - тренировка, 20% - тест):

train_X ,test_X = X[:80,:],X[80:,:]
train_y ,test_y = y[:80,:],y[80:,:]

In [606]:
#заводим функцию сигмоиды
def sigmoid(z) : 
    h = 1 / (1 + np.exp(-z))
    return h 

Обучаем модель

In [607]:


m = train_X.shape[0] # количество строк тренировочной выборки
m_test = test_X.shape[0] # количество строк тестовой выборки

#добавим колонку со смещением
train_X = np.hstack(( np.ones((m,1)) ,train_X)) 
test_X = np.hstack(( np.ones((m_test,1)) ,test_X))

# укажем количество искомых классов
k = 2
n =train_X.shape[1]

# вводим Тету
theta = np.zeros((n,k))

In [608]:
prediction = np.argmax(test_X @ theta, axis=1) 
accuracy = np.mean(prediction == test_y.flatten()) * 100
accuracy

65.0

Так себе обучилась. теперь попробуем оптимизировать, введя градиент

In [609]:
#создадим функцию потерь 
def costFunction(theta, X, y):
    m = X.shape[0]
    h = sigmoid(X @ theta)
    temp1 = np.multiply(y,np.log(h))
    temp2 =np.multiply( (1 - y), np.log(1 - h))
    cost = -(1/m)* np.sum(temp1 + temp2) 
    return cost

Добавим функцию градиентного спуска:

In [610]:
def gradient(theta,X,y):
    m = X.shape[0]
    temp = sigmoid(np.dot(X, theta)) - y
    grad = np.dot(temp.T, X).T / m
    
    return grad

А теперь прогоняем через градиент:

In [595]:
for i in range(k) :
    theta[:,i] = opt.fmin_cg(
        f=costFunction,
        fprime = gradient, # если честно, здесь можгно было обойтись и без градиента, добавив просто train_X вместо и результат был бы не хуже
        x0=theta[:,i],
        args=(train_X,(train_y == i).flatten()),
        maxiter=100
    )

         Current function value: 0.059522
         Iterations: 100
         Function evaluations: 268
         Gradient evaluations: 268
Optimization terminated successfully.
         Current function value: 0.059519
         Iterations: 81
         Function evaluations: 224
         Gradient evaluations: 224


In [597]:
prediction = np.argmax(test_X @ theta, axis=1) 
accuracy = np.mean(prediction == test_y.flatten()) * 100
accuracy

95.0

In [407]:
from matplotlib import pyplot as plt
import numpy as np
from math import ceil, floor

In [411]:
# NM(X, gamma=0.9, lr=0.001)

In [412]:
def linear_interpolation(X, idx):
    idx_min = floor(idx)
    idx_max = ceil(idx)
    if idx_min == idx_max or idx_max >= len(X):
        return X[idx_min]
    elif idx_min < 0:
        return X[idx_max]
    else:
        return X[idx_min] + (idx - idx_min)*X[idx_max]

In [ ]:
def linear_interpolation(X, idx):
    idx_min = floor(idx)
    idx_max = ceil(idx)
    if idx_min == idx_max or idx_max >= len(X):
        return X[idx_min]
    elif idx_min < 0:
        return X[idx_max]
    else:
        return X[idx_min] + (idx - idx_min)*X[idx_max]
# Y1 = []
def NM(X, gamma=0.9, lr=0.25):
    v = 0
    Y1 = []
    for i in range(len(X)):
        v = gamma*v + lr*(linear_interpolation(X, i+gamma*v) if i+gamma*v < len(X) else 0)
        Y1.append(v)
    return np.asarray(Y1)
NM(train_X, gamma=0.9, lr=0.25)